# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../output_data/worldcities_2022-08-03.csv")

df

,Unnamed: 0,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED
0,0,"{'coord': {'lon': -126.8329, 'lat': 65.282}, '...",1659562031,Norman Wells,CA,65.2820,-126.8329,60.84,72.0,75.0,12.66
1,1,"{'coord': {'lon': 25.5701, 'lat': -33.918}, 'w...",1659561930,Port Elizabeth,ZA,-33.9180,25.5701,59.29,72.0,0.0,6.91
2,3,"{'coord': {'lon': -47.3561, 'lat': -0.6136}, '...",1659562186,Salinópolis,BR,-0.6136,-47.3561,80.40,79.0,65.0,12.30
3,4,"{'coord': {'lon': -147.8028, 'lat': 64.8569}, ...",1659562188,College,US,64.8569,-147.8028,68.95,64.0,40.0,4.61
4,5,"{'coord': {'lon': -65.7125, 'lat': 66.1451}, '...",1659562177,Pangnirtung,CA,66.1451,-65.7125,50.00,71.0,75.0,5.75
...,...,...,...,...,...,...,...,...,...,...,...
556,609,"{'coord': {'lon': -1.353, 'lat': 60.3964}, 'we...",1659563260,Brae,GB,60.3964,-1.3530,51.75,81.0,46.0,13.94
557,610,"{'coord': {'lon': -70.1431, 'lat': -20.2208}, ...",1659563262,Iquique,CL,-20.2208,-70.1431,60.84,77.0,6.0,9.89
558,611,"{'coord': {'lon': 28.6617, 'lat': 56.7105}, 'w...",1659563263,Opochka,RU,56.7105,28.6617,59.02,82.0,68.0,1.28
559,612,"{'coord': {'lon': -76.2, 'lat': 24.9}, 'weathe...",1659563265,Rock Sound,BS,24.9000,-76.2000,82.54,72.0,24.0,11.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key) 

In [4]:
locations = df[["LAT", "LNG"]]

weights = df["HUMIDITY"]

In [10]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=weights.max(),
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [11]:
best_cities_df = df.loc[ (df["MAX_TEMP"].between(69, 84)) & (df["HUMIDITY"] < 80) & (df["CLOUDINESS"] < 50), :]
best_cities_df.dropna()
best_cities_df

,Unnamed: 0,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED
57,61,"{'coord': {'lon': -152.4072, 'lat': 57.79}, 'w...",1659561894,Saint Paul Harbor,US,57.7900,-152.4072,69.31,52.0,0.0,12.66
58,62,"{'coord': {'lon': 28.9833, 'lat': 41.0351}, 'w...",1659562280,Istanbul,TR,41.0351,28.9833,74.46,78.0,0.0,11.50
59,63,"{'coord': {'lon': -42.0278, 'lat': -22.9661}, ...",1659562137,Arraial do Cabo,BR,-22.9661,-42.0278,71.46,73.0,20.0,6.91
68,72,"{'coord': {'lon': -139.0333, 'lat': -9.8}, 'we...",1659561914,Atuona,PF,-9.8000,-139.0333,77.25,78.0,16.0,14.03
72,77,"{'coord': {'lon': -159.319, 'lat': 22.0752}, '...",1659561904,Kapaa,US,22.0752,-159.3190,81.30,65.0,40.0,13.80
...,...,...,...,...,...,...,...,...,...,...,...
530,583,"{'coord': {'lon': 100.4517, 'lat': 38.9342}, '...",1659563213,Zhangye,CN,38.9342,100.4517,75.42,51.0,9.0,3.56
545,598,"{'coord': {'lon': 118.7553, 'lat': 30.9525}, '...",1659563240,Xuanzhou,CN,30.9525,118.7553,81.95,79.0,0.0,3.98
554,607,"{'coord': {'lon': -41.3, 'lat': -21.75}, 'weat...",1659563256,Campos dos Goytacazes,BR,-21.7500,-41.3000,71.65,78.0,0.0,13.80
555,608,"{'coord': {'lon': 109.9592, 'lat': 27.5494}, '...",1659563258,Huaihua,CN,27.5494,109.9592,79.12,77.0,30.0,0.96


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = best_cities_df.sample(10)

hotel_df["HOTEL_PLACEID"] = "NULL"
hotel_df["HOTEL_NAME"] = "NULL"
hotel_df["HOTEL_CITY"] = "NULL"
hotel_df["HOTEL_COUNTRY"] = "NULL"

hotel_df = hotel_df.reset_index(drop=True)

In [20]:
# params dictionary 

params = {
    "location": "0,0",
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    
    lat = row["LAT"]
    lng = row["LNG"]

    # change location each iteration while leaving original params in place
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request 
    hotel_data = requests.get(base_url, params=params)

    # convert to json
    hotel_json = hotel_data.json()
   
    try:
        
        hotel_df.loc[index, "HOTEL_PLACEID"] = hotel_json["results"][0]["place_id"]
        
    except (KeyError, IndexError):
        
        print("Missing field/result... skipping.")

In [21]:
cleanhotel_df = hotel_df.loc[ (hotel_df["HOTEL_PLACEID"] != "NULL"), :]

cleanhotel_df = cleanhotel_df.reset_index(drop=True)

cleanhotel_df

,Unnamed: 0,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED,HOTEL_PLACEID,HOTEL_NAME,HOTEL_CITY,HOTEL_COUNTRY
0,545,"{'coord': {'lon': 2.3333, 'lat': 48.9167}, 'we...",1659563145,Arrondissement de Saint-Denis,FR,48.9167,2.3333,80.56,52.0,0.0,11.50,ChIJi-8B97lu5kcR-SinjscaJw4,NULL,NULL,NULL
1,138,"{'coord': {'lon': -48.5164, 'lat': -16.7419}, ...",1659562415,Vianópolis,BR,-16.7419,-48.5164,71.65,34.0,2.0,6.26,ChIJdxw_CobZWJMRWpI1WhF3F3k,NULL,NULL,NULL
2,110,"{'coord': {'lon': 13.765, 'lat': 37.1926}, 'we...",1659562367,Palma di Montechiaro,IT,37.1926,13.7650,79.30,72.0,1.0,0.76,ChIJUXf8nVh9EBMR_OwTDDVMTaI,NULL,NULL,NULL
3,608,"{'coord': {'lon': 109.9592, 'lat': 27.5494}, '...",1659563258,Huaihua,CN,27.5494,109.9592,79.12,77.0,30.0,0.96,ChIJiw8bodFBmDYRMzw4QSTF1XM,NULL,NULL,NULL
4,485,"{'coord': {'lon': -149.3333, 'lat': -17.5333},...",1659563035,Tiarei,PF,-17.5333,-149.3333,82.42,79.0,26.0,26.31,ChIJcycpkDlPmnYRYoOuNcEj0nY,NULL,NULL,NULL
5,240,"{'coord': {'lon': -120.4579, 'lat': 34.6391}, ...",1659562022,Lompoc,US,34.6391,-120.4579,74.68,69.0,0.0,16.11,ChIJyXBkO7Ye7IARBMpzPYOslfw,NULL,NULL,NULL
6,512,"{'coord': {'lon': -56.1381, 'lat': -21.4803}, ...",1659563085,Jardim,BR,-21.4803,-56.1381,80.65,25.0,43.0,3.40,ChIJ48Kn4hkhfZQR_abOpTKVBEk,NULL,NULL,NULL
7,400,"{'coord': {'lon': -151.75, 'lat': -16.4833}, '...",1659562882,Faanui,PF,-16.4833,-151.7500,78.55,75.0,8.0,22.44,ChIJk-lncOC8vXYRc_-EYKBkcSY,NULL,NULL,NULL
8,497,"{'coord': {'lon': -66.8522, 'lat': 10.6112}, '...",1659563057,Caraballeda,VE,10.6112,-66.8522,82.47,78.0,40.0,5.75,ChIJpyM3dUdbKowR_rwloaSwi3M,NULL,NULL,NULL
9,174,"{'coord': {'lon': -155.09, 'lat': 19.7297}, 'w...",1659561910,Hilo,US,19.7297,-155.0900,79.79,74.0,0.0,8.05,ChIJ6VLlrZ5MUnkRILaTYiZiniw,NULL,NULL,NULL


In [22]:
# params dictionary to update each iteration

params = {
    "place_id": "",
    "language": "en",
    "fields": "name,formatted_address",
    "key": g_key
}


for index, row in cleanhotel_df.iterrows():
    
    place_id = row["HOTEL_PLACEID"]
    
    city_name_from_df = cleanhotel_df.iloc[index, 3]
    
    # change location each iteration while leaving original params in place
    
    params["place_id"] = f"{place_id}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/details/json?"

    # make request and print url
    address_data = requests.get(base_url, params=params)

    # convert to json
    address_json = address_data.json()
            
    city_name = ""
    cty_name = ""
    address_list = []
            
    try:
        
        cleanhotel_df.loc[index, "HOTEL_NAME"] = address_json["result"]["name"].strip()
        
        address_list = address_json["result"]["formatted_address"].split(",")
        
        cty_name = address_list[-2].strip() if (address_list[-1].strip()).isnumeric() else address_list[-1].strip()
        
        if city_name_from_df in address_json["result"]["formatted_address"]:
            
            city_name = city_name_from_df
            
        else:
                                    
            city_name = f"{address_list[-3].strip()} {address_list[-2].strip()}"
                                     
        cleanhotel_df.loc[index, "HOTEL_CITY"] = city_name
    
        cleanhotel_df.loc[index, "HOTEL_COUNTRY"] = cty_name
    
    except:
        
        print("Missing field/result... skipping.")    

In [23]:
cleanhotel_df

,Unnamed: 0,JSON,DATE,CITY,COUNTRY,LAT,LNG,MAX_TEMP,HUMIDITY,CLOUDINESS,WIND_SPEED,HOTEL_PLACEID,HOTEL_NAME,HOTEL_CITY,HOTEL_COUNTRY
0,545,"{'coord': {'lon': 2.3333, 'lat': 48.9167}, 'we...",1659563145,Arrondissement de Saint-Denis,FR,48.9167,2.3333,80.56,52.0,0.0,11.50,ChIJi-8B97lu5kcR-SinjscaJw4,ibis Saint-Denis Stade Ouest,20 Rue Jules Saulnier 93200 Saint-Denis,France
1,138,"{'coord': {'lon': -48.5164, 'lat': -16.7419}, ...",1659562415,Vianópolis,BR,-16.7419,-48.5164,71.65,34.0,2.0,6.26,ChIJdxw_CobZWJMRWpI1WhF3F3k,Hotel Regente,Vianópolis,Brazil
2,110,"{'coord': {'lon': 13.765, 'lat': 37.1926}, 'we...",1659562367,Palma di Montechiaro,IT,37.1926,13.7650,79.30,72.0,1.0,0.76,ChIJUXf8nVh9EBMR_OwTDDVMTaI,Chiaramontane Exclusive Resort Terraces,21 92020 Marina di Palma AG,Italy
3,608,"{'coord': {'lon': 109.9592, 'lat': 27.5494}, '...",1659563258,Huaihua,CN,27.5494,109.9592,79.12,77.0,30.0,0.96,ChIJiw8bodFBmDYRMzw4QSTF1XM,Vienna International Hotel,Near Shuntian North Avenue (Opposite City Gove...,H284+H2C鹤城区怀化市湖南省 China
4,485,"{'coord': {'lon': -149.3333, 'lat': -17.5333},...",1659563035,Tiarei,PF,-17.5333,-149.3333,82.42,79.0,26.0,26.31,ChIJcycpkDlPmnYRYoOuNcEj0nY,Le Rocher de Tahiti,FJ6M+5CC Faananu,French Polynesia
5,240,"{'coord': {'lon': -120.4579, 'lat': 34.6391}, ...",1659562022,Lompoc,US,34.6391,-120.4579,74.68,69.0,0.0,16.11,ChIJyXBkO7Ye7IARBMpzPYOslfw,Hilton Garden Inn Lompoc,Lompoc,USA
6,512,"{'coord': {'lon': -56.1381, 'lat': -21.4803}, ...",1659563085,Jardim,BR,-21.4803,-56.1381,80.65,25.0,43.0,3.40,ChIJ48Kn4hkhfZQR_abOpTKVBEk,Hotel Jardim,Jardim,Brazil
7,400,"{'coord': {'lon': -151.75, 'lat': -16.4833}, '...",1659562882,Faanui,PF,-16.4833,-151.7500,78.55,75.0,8.0,22.44,ChIJk-lncOC8vXYRc_-EYKBkcSY,Conrad Bora Bora Nui,BP502 Vaitape Bora Bora 98730,French Polynesia
8,497,"{'coord': {'lon': -66.8522, 'lat': 10.6112}, '...",1659563057,Caraballeda,VE,10.6112,-66.8522,82.47,78.0,40.0,5.75,ChIJpyM3dUdbKowR_rwloaSwi3M,Ole Caribe,Macuto 1164 La Guaira,Venezuela
9,174,"{'coord': {'lon': -155.09, 'lat': 19.7297}, 'w...",1659561910,Hilo,US,19.7297,-155.0900,79.79,74.0,0.0,8.05,ChIJ6VLlrZ5MUnkRILaTYiZiniw,Hilo Hawaiian Hotel,Hilo,USA


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{HOTEL_NAME}</dd>
<dt>City</dt><dd>{HOTEL_CITY}</dd>
<dt>Country</dt><dd>{HOTEL_COUNTRY}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in cleanhotel_df.iterrows()]
locations = cleanhotel_df[["LAT", "LNG"]]

In [25]:
# Add marker layer ontop of heat map

 # Assign the marker layer to a variable
    
markers = gmaps.marker_layer(locations=locations, display_info_box=True, info_box_content=hotel_info)

# Add the layer to the map

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))